<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/trained_models/ru_t5_test(rouge_metrics).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 168 (delta 41), reused 29 (delta 8), pack-reused 76
Receiving objects: 100% (168/168), 43.82 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [ ]:
import json

with open('/content/RuLawSimplification/two_way/two_way.json', 'r', encoding='utf8') as rp:
    a = json.load(rp)

In [ ]:
a[100]

In [ ]:
import pandas as pd

df = pd.DataFrame({'Text': [], 'Comment': []})


for article in a:
    for sujet in article['article text']:
        df = df.append({'Text': sujet['text'], 'Comment': sujet['comment']}, ignore_index=True)


In [ ]:
df

In [ ]:
df.to_csv('two_way_trial.csv', index=False)

In [ ]:
df.shape

In [ ]:
df.iloc[:2500].to_csv('train.csv', index=False, header=None)
df.iloc[2500:2800].to_csv('validation.csv', index=False, header=None)
df.iloc[2800:].to_csv('test.csv', index=False, header=None)

In [ ]:
!pip install Sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 17.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 28.5 MB/s 
     |████████████████████████████████| 6.6 MB 50.0 MB/s 
     |████████████████████████████████| 120 kB 67.7 MB/s 


In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 16.5 MB/s 
     |████████████████████████████████| 115 kB 63.9 MB/s 
     |████████████████████████████████| 212 kB 64.2 MB/s 
     |████████████████████████████████| 127 kB 68.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
pd.read_csv('test.csv', encoding='utf8', sep=',', index_col=False, header=None)

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from datasets import load_dataset


dataset = load_dataset('csv', 
                       data_files={'train': ['train.csv'],
                                   'validation': ['validation.csv'],
                                   'test': ['test.csv']},
                       encoding='utf8',
                       index_col = False,
                       header=None,
                       sep=','
                      )

In [ ]:
print(dataset['train'])
print(dataset['validation'])
print(dataset['test'])

In [ ]:
dataset.shape

In [ ]:
dataset.push_to_hub('marcus2000/twowaydata')

In [ ]:
from datasets import load_dataset

dataset = load_dataset('marcus2000/twowaydata')

Downloading:   0%|          | 0.00/892 [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/marcus2000___parquet/marcus2000--twowaydata-70c3f9955e1f2e89/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset['train']

Dataset({
    features: ['0', '1'],
    num_rows: 2500
})

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"], 2)

,0,1
0,"8. Политическая партия ""НОВЫЕ ЛЮДИ"".",За это время в российской избирательной системе появился ряд новшеств.
1,"2) в части 1 статьи 25: а) в пункте 2 слова ""1, 3, 5, 6 и 7"" заменить цифрами ""1, 3 - 7""; б) пункт 4 изложить в следующей редакции: ""4) поступления сведений об аннулировании вида на жительство пенсионеру - иностранному гражданину или лицу без гражданства в соответствии с Федеральным законом от 25 июля 2002 года N 115-ФЗ ""О правовом положении иностранных граждан в Российской Федерации"" - с 1-го числа месяца, следующего за месяцем, в котором в орган, осуществляющий пенсионное обеспечение, поступили соответствующие сведения;"";","- За это время будут подготовлены и приняты поправки в Трудовой кодекс, которые позволят вести электронный кадровый документооборот не только в рамках эксперимента, сформирована подзаконная нормативная база, а также доработана технологическая платформа ""Работа в России"" с учетом выявленных в ходе эксперимента потребностей""."


In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoConfig


tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruT5-large")



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [ ]:
max_input_length = 100
max_target_length = 100


def preprocess_function(examples):
    inputs = [ex for ex in examples['0']]  # 0 - Текст
    targets = [ex for ex in examples['1']]  # 1 - Комментарий
    model_inputs = tokenizer(inputs, 
                             max_length=max_input_length,
                             truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3541: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 492
    })
    validation: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("sberbank-ai/ruT5-large")

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 16

args = Seq2SeqTrainingArguments("multilingual_t5_model_for_law_simplification",
    evaluation_strategy = "epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    push_to_hub=True,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 116 kB 31.9 MB/s 


In [ ]:
from datasets import load_metric


metric = load_metric("sacrebleu")

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

Cloning https://huggingface.co/marcus2000/multilingual_t5_model_for_law_simplification into local empty directory.
Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2500
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1570
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pa

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,nan,0.093900,7.903300
2,No log,nan,0.093900,7.903300
3,No log,nan,0.093900,7.903300
4,0.000000,nan,0.093900,7.903300
5,0.000000,nan,0.093900,7.903300
6,0.000000,nan,0.093900,7.903300
7,0.000000,nan,0.093900,7.903300
8,0.000000,nan,0.093900,7.903300
9,0.000000,nan,0.093900,7.903300
10,0.000000,nan,0.093900,7.903300


The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and 

TrainOutput(global_step=1570, training_loss=0.0, metrics={'train_runtime': 871.0451, 'train_samples_per_second': 28.701, 'train_steps_per_second': 1.802, 'total_flos': 5850033666048000.0, 'train_loss': 0.0, 'epoch': 10.0})

In [ ]:
predidcted = trainer.predict(tokenized_datasets['test'])

The following columns in the test set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 492
  Batch size = 16


In [ ]:
predidcted

PredictionOutput(predictions=array([[     0, 250099,    260, ...,      0,      0,      0],
       [     0, 250099,    259, ...,      0,      0,      0],
       [     0, 250099,    260, ...,      0,      0,      0],
       ...,
       [     0, 250099,    259, ...,      0,      0,      0],
       [     0, 250099,    259, ...,      0,      0,      0],
       [     0, 250099,    260, ...,      0,      0,      0]]), label_ids=array([[  3835,  54312,   2007, ...,   -100,   -100,   -100],
       [   259,   6835,   3262, ...,   -100,   -100,   -100],
       [  1908,  56364,   6203, ...,   -100,   -100,   -100],
       ...,
       [   259, 109415,  63690, ...,   -100,   -100,   -100],
       [   688,  13890,    259, ...,   -100,   -100,   -100],
       [  3771,   3262,   2527, ...,   -100,   -100,   -100]]), metrics={'test_loss': nan, 'test_bleu': 0.0939, 'test_gen_len': 7.435, 'test_runtime': 15.543, 'test_samples_per_second': 31.654, 'test_steps_per_second': 1.994})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to multilingual_t5_model_for_law_simplification
Configuration saved in multilingual_t5_model_for_law_simplification/config.json
Model weights saved in multilingual_t5_model_for_law_simplification/pytorch_model.bin
tokenizer config file saved in multilingual_t5_model_for_law_simplification/tokenizer_config.json
Special tokens file saved in multilingual_t5_model_for_law_simplification/special_tokens_map.json
Copy vocab file to multilingual_t5_model_for_law_simplification/spiece.model
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/2.17G [00:00<?, ?B/s]

Upload file runs/Sep15_19-55-46_0ab1f4eeae4b/events.out.tfevents.1663271772.0ab1f4eeae4b.70.0:  40%|###9      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/marcus2000/multilingual_t5_model_for_law_simplification
   b4a4ac6..bca9218  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/marcus2000/multilingual_t5_model_for_law_simplification
   b4a4ac6..bca9218  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Bleu', 'type': 'bleu', 'value': 0.0939}]}
To https://huggingface.co/marcus2000/multilingual_t5_model_for_law_simplification
   bca9218..c5eb755  main -> main

   bca9218..c5eb755  main -> main



'https://huggingface.co/marcus2000/multilingual_t5_model_for_law_simplification/commit/bca9218d4ff16dcd8b849938a8a8bf498d5c4806'

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 7.0 MB/s 


In [ ]:
import evaluate
# evaluate.list_evaluation_modules(module_type='metric', with_details=True)

In [ ]:
metric_value = 8.1  # value obtained from https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/stt_en_conformer_ctc_small 

evaluate.push_to_hub(
    model_id='marcus2000/fine_tuned_t5_model',
    task_type="automatic-speech-recognition",
    dataset_type="librispeech_asr",
    dataset_name="Librispeech (clean)",
    metric_type="wer",
    metric_name="WER",
    dataset_split="test",  # corresponds to test-clean set
    dataset_config="other",  # corresponds to test-clean set
    dataset_args=dict(language="en"),  # metadata for dataset
    # the actual score obtained by the model
    metric_value=metric_value,
)

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

'https://huggingface.co/marcus2000/fine_tuned_t5_model/blob/main/README.md'